# Trabalho de Motores

## 1. Ensaio de Fluxo

### 1.1 Áreas

#### 1.1.1 Área de cortina $\times$ Levante das válvulas

#### 1.1.2 Área de garganta

### 1.2 Coeficiente de descarga ($C_d$)

#### 1.2.1 $C_d$ considerando área de garganta

#### 1.2.1 $C_d$ considerando área mínima

## 2. Ensaio Dinamométrico

### 2.1 Pressão média $\times$ rotação

### 2.2 Torque $\times$ rotação

### 2.3 Potência $\times$ rotação

### 2.4 Carga lateral do pistão $\times$ rotação
- considerando ângulo de 90o e PME

### 2.5 Velocidade

#### 2.5.1 Velocidade média $\times$ rotação

#### 2.5.2 Velocidade máxima $\times$ rotação

#### 2.5.3 Velocidade máxima com torque máximo

#### 2.5.4 Velocidade máxima com potência máximo

### 2.6 Comportamento da pressão de Boost (admissão) $\times$ rotação
- verificar influência da pressão de Boost

###  2.7 Índice de mach na válvula de admissão em condição de potência e torque máximo

####  2.7.1 Índice de mach com potência máxima

####  2.7.2 Índice de mach com torque máxima